In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from scipy.optimize import curve_fit

In [2]:
directory = directory_project
exp = get_exp((39,269,329),directory)

begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00


In [3]:
exp.load_compressed_skel()

In [3]:
RH, BAS, max_speeds, total_growths, widths_sp, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)

failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure 

In [5]:
exp_clean = exp

In [5]:
anchors = [430,111,374,26]

In [76]:
plt.close('all')
exp_clean.plot([-1],[anchors])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [185]:
exp_clean.plot_raw(-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
plt.close('all')
index = 0
Node(anchors[index],exp).show_source_image(-1,-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
index = 0
t0 = 0
pos = Node(anchors[index],exp).pos(t0)
x1,y1,_,_,_ = find_image_pos(exp, pos[0],pos[1],t0)

In [78]:
index = 3
t0 = 0
pos = Node(anchors[index],exp).pos(t0)
x2,y2,_,_,_ = find_image_pos(exp, pos[0],pos[1],t0)

In [81]:
x0 = 92.007
y0 = 26.928
(x2-x1)*1.725,(y2-y1)*1.725

(1236.825, -7953.975)

In [7]:
number_face=[2+len(g.edges)-len(g.nodes) for g in exp_clean.nx_graph]
number_anas_theory = [number_face[i+1]-number_face[i] for i in range(len(number_face)-2)]

In [8]:
hyph_anas_tip_hyph = [hyphat for hyphat in exp_clean.hyphaes if len(hyphat.ts)>=2 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]
hyph_anas_tip_tip = []
potential = []
for hyph in exp_clean.hyphaes:
    if len(hyph.ts)>=2 and hyph.end.degree(hyph.ts[-1])==1 and hyph.end.ts()[-1]!=len(exp_clean.nx_graph)-1 and not np.all([hyph.get_length_pixel(t)<=20 for t in hyph.ts]):
        potential.append(hyph)
for hyph in potential:
    t0 = hyph.ts[-1]
    for hyph2 in potential:
        if hyph2.ts[-1] == t0 and hyph!=hyph2:
            vector = (hyph2.end.pos(t0)-hyph.end.pos(t0))/np.linalg.norm(hyph2.end.pos(t0)-hyph.end.pos(t0))
            vertical_vector=np.array([-1,0])
            dot_product = np.dot(vertical_vector,vector)
            if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
                angle = np.arccos(dot_product)/(2*np.pi)*360
            else:
                angle = -np.arccos(dot_product)/(2*np.pi)*360
            score = np.cos((angle-(180+hyph.end.edges(t0)[0].orientation_begin(t0,30)))/360*2*np.pi)+np.cos((360+angle-hyph2.end.edges(t0)[0].orientation_begin(t0,30))/360*2*np.pi)
            if np.linalg.norm(hyph2.end.pos(t0)-hyph.end.pos(t0))<=500 and score>=0.5:
                hyph_anas_tip_tip.append((hyph,hyph2,t0))

In [9]:
def get_time_anas_tip_hyph(anas_tip_hyph):
    for t in anas_tip_hyph.ts:
        if anas_tip_hyph.end.degree(t)>=3:
            return(t-1)
def get_time_anas_tip_tip(anas_tip_tip):
    return(anas_tip_tip.ts[-1])

In [10]:
anas_tip_hyph_observed = np.array([len([hyph for hyph in hyph_anas_tip_hyph if get_time_anas_tip_hyph(hyph)==t]) for t in range(len(exp_clean.nx_graph)-2)])

In [11]:
anas_tip_tip_observed = np.array([len([hyphs for hyphs in hyph_anas_tip_tip if hyphs[2]==t])//2 for t in range(len(exp_clean.nx_graph)-2)])

In [12]:
times = [get_time(exp,0,t) for t in range(exp.ts)][:-2]

In [33]:
lapse_anas = []
for hyph in hyph_anas_tip_hyph:
    if hyph in RH:
        for t in hyph.ts:
            if hyph.end.degree(t)>=3:
                time = get_time(exp,hyph.ts[0],t)
                lapse_anas.append((hyph.ts[0],time))
                break

In [34]:
lapses = [lapse for t,lapse in lapse_anas]

In [36]:
len([rh for rh in RH if rh in hyph_anas_tip_hyph]),len(RH)

(113, 333)

In [35]:
np.mean(lapses)

6.532890855457228

In [29]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(lapses,20,density= True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([0.21396413, 0.08660453, 0.0649534 , 0.02165113, 0.01146236,
        0.00764158, 0.00382079, 0.        , 0.00764158, 0.0012736 ,
        0.0012736 , 0.        , 0.00254719, 0.        , 0.        ,
        0.        , 0.0012736 , 0.        , 0.        , 0.0012736 ]),
 array([ 0.        ,  2.35083333,  4.70166667,  7.0525    ,  9.40333333,
        11.75416667, 14.105     , 16.45583333, 18.80666667, 21.1575    ,
        23.50833333, 25.85916667, 28.21      , 30.56083333, 32.91166667,
        35.2625    , 37.61333333, 39.96416667, 42.315     , 44.66583333,
        47.01666667]),
 <a list of 20 Patch objects>)

In [13]:
fig = plt.figure()
ax = fig.add_subplot(111)
total_anastomosis_observed = np.cumsum(np.array(anas_tip_hyph_observed)+np.array(anas_tip_tip_observed))
total_anastomosis_theory = np.cumsum(number_anas_theory)

def func2(t,alpha,C,d):
    return(C*(np.exp(alpha*(t)))+d)
popt1, pcov = curve_fit(func2, times, total_anastomosis_observed,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
popt2, pcov = curve_fit(func2, times, total_anastomosis_theory,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])

ax.scatter(times,total_anastomosis_observed, label = 'Total number of anastomosis observed')
ax.scatter(times,total_anastomosis_theory, label = 'Theroretical number of anastomosis')
ax.plot(times,func2(np.array(times),*popt1),label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='blue')
ax.plot(times,func2(np.array(times),*popt2),label = f'exponential fit : alpha= {"{:.2f}".format(popt2[0]*24)}.day-1',color='orange')
ax.set_xlabel('time (h)')
ax.set_ylabel('total number of anastomosis')
plt.legend()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
anas_tip_tip_observed, anas_tip_hyph_observed

(array([0, 2, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 2, 1, 0, 0, 2, 1, 0, 0, 0, 1, 2, 0, 1, 0, 1, 0, 2, 1, 0, 0,
        3, 2, 2, 0, 3, 0, 2, 1, 2, 7, 1, 3, 1, 4, 7]),
 array([ 3,  2,  3,  2,  1,  0,  1,  1,  2,  0,  4,  1,  2,  4,  2,  5,  8,
         3,  2,  3,  3,  5,  2,  2,  5,  2,  7,  5,  4,  5,  6,  7,  6, 10,
         7,  9,  8,  5,  9,  7,  5,  3, 10,  5, 12,  9,  6, 11,  9, 12,  8,
         7, 11, 10,  7, 10, 11, 12, 13]))

In [15]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.array(anas_tip_hyph_observed)+np.array(anas_tip_tip_observed))
ax.plot(number_anas_theory)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
def func(t,alpha,C):
    return(C*(np.exp(alpha*(t))))
end = 40
begin = 10
coefs = np.polyfit(times[begin:end],lengths[begin:end],2)
popt0, pcov = curve_fit(func, times[begin:end], lengths[begin:end],bounds = ([0,0],2*[np.inf]),p0=[1,1])
p = np.poly1d(coefs)

In [21]:
plt.close('all')
hyph,hyph2,t0 = choice(hyph_anas_tip_tip)

In [22]:
plt.close('all')
exp_clean.plot([t0,t0+1],[[hyph.end.label,hyph.root.label,hyph2.end.label,hyph2.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [262]:
plt.close('all')
hyph = choice(hyph_anas_tip_hyph)
t0 = get_time_anas_tip_hyph(hyph)
# exp_clean.plot([t0,t0+1],[[hyph.end.label,hyph.root.label]]*2)

In [265]:
hyph.end.show_source_image(t0+1,t0+1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
Node(11214,exp).show_source_image(28,28)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [266]:
x2,y2,_,_,_ = find_image_pos(exp, hyph.end.pos(t0+1)[0],hyph.end.pos(t0+1)[1],t0+1)

In [267]:
x0 = 46640
y0 = 52420
(x2-x1)*1.725,(y2-y1)*1.725

(-1930.275, -8909.625)

In [10]:
def find_image_pos(exp, xs, ys, t, local=False):
    date = exp.dates[t]
    directory_name = get_dirname(date, exp.plate)
    path_snap = exp.directory + directory_name
    path_tile = path_snap + "/Img/TileConfiguration.txt.registered"
    skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")
    Rot = skel["R"]
    trans = skel["t"]
    rottrans = np.dot(np.linalg.inv(Rot), np.array([xs, ys] - trans))
    ys, xs = round(rottrans[0]), round(rottrans[1])
    tileconfig = pd.read_table(
        path_tile,
        sep=";",
        skiprows=4,
        header=None,
        converters={2: ast.literal_eval},
        skipinitialspace=True,
    )
    xs_yss = list(tileconfig[2])
    xes = [xs_ys[0] for xs_ys in xs_yss]
    yes = [xs_ys[1] for xs_ys in xs_yss]
    cmin = np.min(xes)
    cmax = np.max(xes)
    rmin = np.min(yes)
    rmax = np.max(yes)
    ximg = xs
    yimg = ys

    def find(xsub, ysub, x, y):
        indexes = []
        for i in range(len(xsub)):
            if (
                x >= xsub[i] - cmin
                and x < xsub[i] - cmin + 4096
                and y >= ysub[i] - rmin
                and y < ysub[i] - rmin + 3000
            ):
                indexes.append(i)
        return indexes

    indsImg = find(xes, yes, ximg, yimg)
    possImg = [
        ximg - np.array(xes)[indsImg] + cmin + 1,
        yimg - np.array(yes)[indsImg] + rmin + 1,
    ]
    paths = []
    for index in indsImg:
        name = tileconfig[0][index]
        imname = '/Img/'+name.split('/')[-1]
        directory_name = get_dirname(date, exp.plate)
        path  = exp.directory + directory_name + imname
        pipeline.paths.append(path)
    ims = [imageio.imread(path) for path in paths]
    return (xs,ys, indsImg[0],possImg[0][0],possImg[1][0])